<a href="https://colab.research.google.com/github/KKeun-B/learning/blob/main/2020/Python_Hyperbolic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import os
import shutil
import numpy as np

In [2]:
class Hyperbolic:
    PI = math.pi

    def __init__(self):
        self.scheme = 'Ex_Upwind'
        print("Solve Explicit Upwind")
        self.Scheme_name = "Explicit_Upwind"

    def Initialize(self):
        self.dx = 5.0
        self.dt = 0.02
        self.end_t = 1.0
        self.L = 400
        self.alpha = 250.0

    def CalDeltaT(self):
        self.dt = self.CFL * self.dx / self.alpha
        self.c = self.CFL * self.CFL

    def Dir_Write(self):
        self.dirname = "Hyperbolic,{0},c={1}".format(self.Scheme_name, self.CFL)
        path = os.getcwd()
        self.dirname = os.path.join(path, self.dirname)
        if os.path.isdir(self.dirname):
            shutil.rmtree(self.dirname)  # 디렉토리가 존재하면 삭제하고 다시 계산# return # 디렉토리가 존재하면 덮어쓰기 #
        os.mkdir(self.dirname)
        return

    def Para_Write(self):
        filename = "{0}/{1}, c = {2}.csv.".format(self.dirname, self.Scheme_name, self.CFL)
        filename = "%s%.6f"%(filename, self.time)
        file = open(filename, 'w')
        file.write("X,Y,Z,Velocity\n")
        for i in range(self.N):
            data = "%3.3f,%3.3f,%3.3f,%3.3f\n" % (float(i*self.dx), 0.0, 0.0, self.U[i])
            file.write(data)
        file.close()

    def Initial_Condition(self):
        for i in range(int(170.0/self.dx) , int(230.0/self.dx+1)):
            self.U[i] = abs(100.0 * (math.sin(self.PI*(i*self.dx - 50.0)/60.0)))
            self.Unew[i] = abs(100.0 * (math.sin(self.PI*(i*self.dx - 50.0)/60.0)))
            self.Uold[i] = abs(100.0 * (math.sin(self.PI*(i*self.dx - 50.0)/60.0)))

    def Boundary_Condition(self):
        self.U[0] = 0.0
        self.Unew[0] = 0.0
        self.Uold[0] = 0.0
        self.U[self.N-1] = 0.0
        self.Unew[self.N-1] = 0.0
        self.Uold[self.N-1] = 0.0

    def Time_Marching(self):
        self.Uold = self.U.copy()
        self.U = self.Unew.copy()

    def Ex_Upwind(self):
        for i in range(1, self.N-1):
            self.Unew[i] = 2 * (1 - self.c) * self.U[i] - self.Uold[i] + self.c * (self.U[i+1] + self.U[i-1])


    def Main(self):
        self.Initialize()
        CFL = [1.0, 0.5, 0.25]
        for i in CFL:
            self.time = 0.0
            self.CFL = i
            self.CalDeltaT()
            self.N = int(float(self.L)/self.dx)
            self.U = np.zeros(self.N)
            self.Unew = np.zeros(self.N)
            self.Uold = np.zeros(self.N)
            self.Initial_Condition()
            self.Boundary_Condition()
            self.iter = int(self.end_t/self.dt)
            self.Dir_Write()
            for j in range(self.iter+1):
                if float(j) % (1.0 / self.CFL) == 0:
                    self.Para_Write()
                self.Ex_Upwind()
                self.Time_Marching()
                self.Boundary_Condition()
                self.time += self.dt
                print("\rCFL = %3.3f, time = %.6f" % (self.CFL, self.time), end="")
            print()

In [3]:
PyCompute = Hyperbolic()
PyCompute.Main()

Solve Explicit Upwind
CFL = 1.000, time = 1.020000
CFL = 0.500, time = 1.010000
CFL = 0.250, time = 1.005000


In [4]:
!zip -r -qq Hyperbolic,Explicit_Upwind,c=1.0.zip Hyperbolic,Explicit_Upwind,c=1.0
!zip -r -qq Hyperbolic,Explicit_Upwind,c=0.5.zip Hyperbolic,Explicit_Upwind,c=0.5
!zip -r -qq Hyperbolic,Explicit_Upwind,c=0.25.zip Hyperbolic,Explicit_Upwind,c=0.25